# 简介

这是一款名为‘FunctionAgentWithRetrieval检索增强小助手’的演示应用，展示了如何结合LangChain的工具和ERNIEBot的function agent来回答用户的专业知识问题。首先，应用通过构建FaissSearch在知识库中检索相关内容，然后评估检索内容与问题的相关度。如果内容与问题高度相关，大模型将采用检索增强方式回答问题；否则，大模型将调用工具列表中的工具回答用户的问题。这种设计不仅扩展了大模型的专业领域知识，还保持了其在领域知识之外的通用对话能力

构建流程如下：

# 1. 导入第三方库

主要是在导入一些必要的Python库和模块，以便实现FunctionAgentWithRetrival的功能。
+ os: Python的标准库，用于与操作系统进行交互，如读写文件、管理路径等。
+ SpacyTextSplitter: 一个文本分割工具，用于将文本分割为更小的部分，如句子或短语。
+ FAISS: 用于向量存储的模块，是用于存储和检索经过嵌入处理的文本或图像的向量表示。
+ PyPDFDirectoryLoader: 用于从PDF文件中加载数据的工具。
+ ErnieEmbeddings: 用于文本嵌入的工具，将文本转换为可以在模型中使用的向量表示。
+ FunctionAgent: 这个类实现function calling功能的Agent的类，如问答、对话等。
+ WholeMemory: 用于存储和管理代理的记忆的类。
+ ERNIEBot: 实现ERNIE Bot的主要类，包含了实现对话功能的主要逻辑。
+ erniebot: 导入erniebot模块的语句，erniebot是一个包含ERNIE Bot实现的主要代码库。
+ cosine_similarity: 用于计算余弦相似度的函数
这里使用EB_AGENT_ACCESS_TOKEN, 申请地址请参考[accessToken](https://aistudio.baidu.com/index/accessToken)

In [1]:
import os
from erniebot_agent.agents.function_agent_with_retrieval import FunctionAgentWithRetrieval
from erniebot_agent.memory.whole_memory import WholeMemory
from erniebot_agent.chat_models.erniebot import ERNIEBot
from erniebot_agent.tools.calculator_tool import CalculatorTool
import erniebot
from langchain.text_splitter import SpacyTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader
from erniebot_agent.extensions.langchain.embeddings import ErnieEmbeddings
from sklearn.metrics.pairwise import cosine_similarity

os.environ["EB_AGENT_ACCESS_TOKEN"] = "your access token"

# 2. 预处理

## 2.1 下载数据集

In [22]:
! wget https://paddlenlp.bj.bcebos.com/datasets/examples/construction_regulations.tar
! tar xvf construction_regulations.tar

--2023-12-25 07:14:37--  https://paddlenlp.bj.bcebos.com/datasets/examples/construction_regulations.tar
Resolving paddlenlp.bj.bcebos.com (paddlenlp.bj.bcebos.com)... 36.110.192.178, 2409:8c04:1001:1002:0:ff:b001:368a
Connecting to paddlenlp.bj.bcebos.com (paddlenlp.bj.bcebos.com)|36.110.192.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1239040 (1.2M) [application/x-tar]
Saving to: ‘construction_regulations.tar’

construction_regula 100%[===================>]   1.18M  1.16MB/s    in 1.0s    

2023-12-25 07:14:38 (1.16 MB/s) - ‘construction_regulations.tar’ saved [1239040/1239040]

construction_regulations/
construction_regulations/城市管理执法办法.pdf
construction_regulations/建筑工程设计招标投标管理办法.pdf
construction_regulations/建筑业企业资质管理规定.pdf
construction_regulations/城市照明管理规定.pdf
construction_regulations/城市设计管理办法.pdf
construction_regulations/建筑工程施工发包与承包计价管理办法.pdf
construction_regulations/市政公用设施抗灾设防管理规定.pdf


## 2.2 配置 ERNIE BOT API

In [ ]:
erniebot.api_type = "aistudio"

## 2.3 构建FaissSearch
这段代码定义了一个名为 FaissSearch 的类，用于在一个数据库中执行相似性搜索。这个类的主要功能是使用 FAISS (Facebook AI Similarity Search) 库或类似技术，对文档数据库进行高效的相似性搜索。它首先找到与查询最相似的文档，然后计算每个文档与查询的余弦相似度，最后返回包含内容、相似度得分和文档标题的搜索结果。

In [3]:
class FaissSearch:
    def __init__(self, db):
        # 类的初始化方法，接收一个数据库实例并将其存储在类的实例变量 self.db 中
        self.db = db

    def search(self, query: str, top_k: int = 10, **kwargs):
        # 定义一个搜索方法，接受一个查询字符串 'query' 和一个整数 'top_k'，默认为 10
        docs = self.db.similarity_search(query, top_k)
        # 调用数据库的 similarity_search 方法来获取与查询最相关的文档
        para_result = embeddings.embed_documents([i.page_content for i in docs])
        # 对获取的文档内容进行嵌入（embedding），以便进行相似性比较
        query_result = embeddings.embed_query(query)
        # 对查询字符串也进行嵌入
        similarities = cosine_similarity([query_result], para_result).reshape((-1,))
        # 计算查询嵌入和文档嵌入之间的余弦相似度
        retrieval_results = []
        for index, doc in enumerate(docs):
            retrieval_results.append(
                {"content": doc.page_content, "score": similarities[index], "title": doc.metadata["source"]}
            )
        # 遍历每个文档，将内容、相似度得分和来源标题作为字典添加到结果列表中
        return retrieval_results  # 返回包含搜索结果的列表

# 3.FunctionAgentWithRetrieval
## 3.1 建索引库

这段代码主要功能是用于创建或加载一个FAISS索引来进行文档相似度匹配。首先，创建一个ErnieEmbeddings对象，用于生成文档的嵌入向量。aistudio_access_token是访问AI Studio的令牌，chunk_size是用于嵌入的文档块的数量。

In [4]:
aistudio_access_token = os.environ.get("EB_AGENT_ACCESS_TOKEN", "")
embeddings = ErnieEmbeddings(aistudio_access_token=aistudio_access_token, chunk_size=16)

接下来利用ErnieEmbeddings来抽取向量构建索引。
+ 如果FAISS索引文件已经存在，就使用FAISS.load_local方法加载这个索引，这个索引文件的名字就是定义的faiss_name。
+ 如果FAISS索引不存在，则需要建索引。
    + 第一步，使用PyPDFDirectoryLoader来从"construction_regulations"这个文件夹中加载PDF文档。
    + 第二步，使用SpacyTextSplitter来将加载的文档分割成更小的部分，以便于生成嵌入向量。这个分割器主要用于中文文本，因为这里使用的pipeline是'zh_core_web_sm'，如果是初次运行，需要安装spacy并运行`python -m spacy download zh_core_web_sm`来下载中文分句模型。
    + 第三步，通过分割后的文档创建一个新的FAISS索引，并将这个索引保存为之前定义的faiss_name。

In [5]:
embeddings = ErnieEmbeddings(aistudio_access_token=aistudio_access_token, chunk_size=16)
faiss_name = "faiss_index"
if os.path.exists(faiss_name):
    db = FAISS.load_local(faiss_name, embeddings)
else:
    loader = PyPDFDirectoryLoader("construction_regulations")
    documents = loader.load()
    text_splitter = SpacyTextSplitter(pipeline="zh_core_web_sm", chunk_size=320, chunk_overlap=0)
    docs = text_splitter.split_documents(documents)
    db = FAISS.from_documents(docs, embeddings)
    db.save_local(faiss_name)

Created a chunk of size 408, which is longer than the specified 320
Retrying requests: Attempt 1 ended with: <Future at 0x7f49af5a8970 state=finished raised RateLimitError>
Retrying requests: Attempt 1 ended with: <Future at 0x7f49af5aa680 state=finished raised RateLimitError>
Retrying requests: Attempt 1 ended with: <Future at 0x7f49af5a9120 state=finished raised RateLimitError>
Retrying requests: Attempt 1 ended with: <Future at 0x7f49af5a9f30 state=finished raised RateLimitError>
Retrying requests: Attempt 1 ended with: <Future at 0x7f49af5a8460 state=finished raised RateLimitError>
Retrying requests: Attempt 1 ended with: <Future at 0x7f49af5a9de0 state=finished raised RateLimitError>
Retrying requests: Attempt 1 ended with: <Future at 0x7f49af5a82b0 state=finished raised RateLimitError>
Retrying requests: Attempt 1 ended with: <Future at 0x7f49af5aa860 state=finished raised RateLimitError>
Retrying requests: Attempt 1 ended with: <Future at 0x7f49af5a9090 state=finished raised Rat

以下代码是使用FaissSearc进行搜索的一个例子，流程大致如下：
1. 创建FaissSearch对象，并传入数据库对象db。
2. 调用FaissSearch对象的search函数，并传入查询字符串"城市管理执法主管部门的职责是什么？"。
3. 将搜索结果以格式化的形式进行打印，这里使用了pprint模块进行美化打印。最终的结果存储在变量res中。

In [7]:
faiss_search = FaissSearch(db=db)
res = faiss_search.search(query="城市管理执法主管部门的职责是什么？")
from pprint import pprint

pprint(res)

Retrying requests: Attempt 1 ended with: <Future at 0x7f4b22a22b60 state=finished raised RateLimitError>
Retrying requests: Attempt 1 ended with: <Future at 0x7f49ae5d4a30 state=finished raised RateLimitError>


[{'content': '第十条 城市管理执法主管部门依法相对集中行使行政处罚权的， 可以实施法律法规规定的与行政处罚权相关的行政强制措施。',
  'score': 0.6833730421296541,
  'title': 'construction_regulations/城市管理执法办法.pdf'},
 {'content': '第六条 城市管理执法主管部门应当加强城市管理法律法规规章的宣传普及工作，增强全民守法意识，共同维护城市管理秩序。\n'
             '\n'
             '  \n'
             '\n'
             '第七条 城市管理执法主管部门应当积极为公众监督城市管理执法活动提供条件。',
  'score': 0.645987793263916,
  'title': 'construction_regulations/城市管理执法办法.pdf'},
 {'content': '其他违反法律法规和本办法规定的。   第四十条 '
             '非城市管理执法人员着城市管理执法制式服装的，城市管理执法主管部门应当予以纠正，依法追究法律责任。',
  'score': 0.628888526926951,
  'title': 'construction_regulations/城市管理执法办法.pdf'},
 {'content': '第三章 执法主体     第十三条 城市管理执法主管部门按照权责清晰、事权统一、精简效能的原则设置执法队伍。   第十四条 '
             '直辖市、 设区的市城市管理执法推行市级执法或者区级执法。   '
             '直辖市、设区的市的城市管理执法事项，市辖区人民政府城市管理执法主管部门能够承担的，可以实行区级执法。   直辖市、 '
             '设区的市人民政府城市管理执法主管部门可以承担跨区域和重大复杂违法案件的查处。',
  'score': 0.6045487776924248,
  'title': 'construction_regulations/城市管理执法办法.pdf'},
 {'content': '第五章 执法规范     第二十五条 '
             '城

## 3.2 构建FunctionAgentWithRetrieval
FunctionAgentWithRetrieval可以是否进行检索增强回答用户问题，以下是其示例。
### 3.2.1 检索增强

In [8]:
# 创建一个ERNIEBot实例，使用"ernie-3.5"模型。
llm = ERNIEBot(model="ernie-3.5")
# 创建一个WholeMemory实例。这是一个用于存储对话历史和上下文信息的类，有助于模型理解和持续对话。
memory = WholeMemory()
# 创建一个CalculatorTool实例。这是一个计算器工具。
calculator_tool = CalculatorTool()
# 创建一个FunctionAgentWithRetrieval实例。这个代理将使用上面创建的ERNIEBot模型、WholeMemory和faiss_search，同时传入了一个名为calculator_tool的工具。
agent = FunctionAgentWithRetrieval(
    llm=llm, tools=[calculator_tool], memory=memory, knowledge_base=faiss_search, threshold=0.5
)

In [11]:
# 定义一个查询字符串，这个查询是关于"城乡建设部规章中，城市管理执法第三章，第十三条"的内容。
query = "城乡建设部规章中，城市管理执法第三章，第十三条是什么？"
# 使用agent的async_run方法来异步执行查询。由于这是异步操作，因此需要使用'await'关键字。
response = await agent.async_run(query)
messages = response.chat_history
for item in messages:
    print(item.to_dict())

Retrying requests: Attempt 1 ended with: <Future at 0x7fd1708d3d90 state=finished raised RateLimitError>
Retrying requests: Attempt 1 ended with: <Future at 0x7fd1708d3ac0 state=finished raised RateLimitError>


{'role': 'user', 'content': '检索结果:\n\n    第1个段落: 住房和城乡建设部规章 \n   X住房和城乡建设部发布  \n-\n\n    第2个段落: 住房和城乡建设部规章 \n   X住房和城乡建设部发布  \n-\n\n    第3个段落: 第十条 城市管理执法主管部门依法相对集中行使行政处罚权的， 可以实施法律法规规定的与行政处罚权相关的行政强制措施。\n\n检索语句: 城乡建设部规章中，城市管理执法第三章，第十三条是什么？\n请根据以上检索结果回答检索语句的问题'}
{'role': 'assistant', 'content': '根据以上检索结果，没有找到关于“城乡建设部规章中，城市管理执法第三章，第十三条”的具体内容。可能是因为该规章的具体内容没有被提供或者是在提供的段落中没有包含相关信息。建议您通过其他渠道查询该规章的完整内容，以获取所需的具体条款。', 'function_call': None, 'search_info': {'results': [{'index': 1, 'url': '', 'title': 'construction_regulations/市政公用设施抗灾设防管理规定.pdf'}, {'index': 2, 'url': '', 'title': 'construction_regulations/市政公用设施抗灾设防管理规定.pdf'}, {'index': 3, 'url': '', 'title': 'construction_regulations/城市管理执法办法.pdf'}]}}


### 3.2.2 调用tool示例
在这个例子中，FunctionAgentWithRetrieval没有利用其检索信息，而是直接调用计算器工具回答用户的问题

In [9]:
# 定义一个查询字符串，这个查询是关于"11-2="的内容。
response = await agent._async_run("11-2=")
# 使用agent的async_run方法来异步执行查询。由于这是异步操作，因此需要使用'await'关键字。
messages = response.chat_history
for item in messages:
    print(item.to_dict())

Retrying requests: Attempt 1 ended with: <Future at 0x7f49ae701ab0 state=finished raised RateLimitError>
Retrying requests: Attempt 1 ended with: <Future at 0x7f49ae701ab0 state=finished raised RateLimitError>


{'role': 'user', 'content': '11-2='}
{'role': 'assistant', 'content': '', 'function_call': {'name': 'CalculatorTool', 'thoughts': '用户想要计算11减去2的结果，我可以使用CalculatorTool工具来完成这个任务。', 'arguments': '{"math_formula":"11-2"}'}, 'search_info': None}
{'role': 'function', 'name': 'CalculatorTool', 'content': '{"formula_result": 9}'}
{'role': 'assistant', 'content': '根据您的请求，11减去2的结果是9。如果您还有其他问题或需要执行其他数学公式，请随时告诉我。', 'function_call': None, 'search_info': None}
